# Analisi del Sentiment nel Customer Care

Questo notebook illustra come implementare un sistema di analisi del sentiment per le comunicazioni dei clienti utilizzando tecniche di Natural Language Processing (NLP). Ci concentreremo su un caso d'uso tipico del customer care: l'identificazione automatica del tono emotivo nei messaggi dei clienti.

## Obiettivi del Notebook

- Creare un dataset di esempio di comunicazioni di customer care
- Preprocessare i testi per l'analisi del sentiment
- Implementare e confrontare diversi approcci di analisi del sentiment
- Valutare le performance dei modelli
- Applicare il modello a nuovi dati
- Visualizzare e interpretare i risultati

## Caso d'uso: Analisi del Sentiment nel Customer Care

L'analisi del sentiment nel customer care permette di:

- Identificare rapidamente clienti insoddisfatti che potrebbero richiedere attenzione immediata
- Monitorare la soddisfazione generale dei clienti nel tempo
- Valutare l'efficacia degli operatori nel migliorare il sentiment durante le interazioni
- Identificare prodotti, servizi o processi che generano feedback negativi
- Riconoscere e valorizzare feedback positivi

In questo notebook, svilupperemo un sistema che può classificare le comunicazioni dei clienti in tre categorie di sentiment:
- Positivo
- Neutro
- Negativo

## 1. Setup e Importazione delle Librerie

In [ ]:
# Importazione delle librerie necessarie
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Librerie per il preprocessing del testo
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

# Librerie per la feature extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Librerie per la modellazione
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# Librerie per la valutazione
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Librerie per l'analisi del sentiment
from textblob import TextBlob
import nltk.sentiment.vader as vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Librerie per modelli avanzati
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# Impostazioni di visualizzazione
plt.style.use('ggplot')
sns.set(style='whitegrid')

# Download delle risorse NLTK necessarie
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

## 2. Creazione di un Dataset di Esempio

Per questo notebook, creeremo un dataset sintetico di comunicazioni di customer care con sentiment etichettati. In un contesto reale, utilizzeresti dati storici delle comunicazioni con i clienti, opportunamente anonimizzati e etichettati.

In [ ]:
# Creazione di un dataset di esempio
data = {
    'testo': [
        # Sentiment positivo
        "Grazie mille per la rapida risposta! Il problema è stato risolto perfettamente.",
        "Sono molto soddisfatto del vostro servizio clienti, l'operatore è stato gentilissimo e competente.",
        "Ottimo lavoro! Il tecnico ha risolto il problema in pochissimo tempo e con grande professionalità.",
        "Volevo complimentarmi per l'efficienza del vostro servizio, non mi aspettavo una soluzione così rapida.",
        "La nuova app è fantastica, molto più intuitiva e veloce della precedente. Complimenti!",
        "Esperienza d'acquisto eccellente, dalla consulenza alla consegna tutto è stato impeccabile.",
        "Il vostro operatore Mario è stato estremamente disponibile e paziente, merita un riconoscimento.",
        "Finalmente un servizio clienti che funziona! Problema risolto al primo contatto, incredibile.",
        "Sono rimasto piacevolmente sorpreso dalla qualità del prodotto, supera le mie aspettative.",
        "La procedura di reso è stata semplicissima e il rimborso è arrivato in tempi record. Grazie!",
        "Apprezzo molto la trasparenza e l'onestà con cui avete gestito il mio reclamo.",
        "Il vostro servizio clienti è il migliore con cui abbia mai avuto a che fare, continuate così!",
        "Grazie per aver risolto il problema anche se era fuori garanzia, avete guadagnato un cliente fedele.",
        "La qualità del supporto tecnico è eccezionale, l'operatore ha risolto un problema che mi affliggeva da mesi.",
        "Sono felicissimo della nuova offerta, il rapporto qualità-prezzo è imbattibile.",
        
        # Sentiment neutro
        "Vorrei informazioni sul processo di attivazione del nuovo servizio.",
        "Ho ricevuto la fattura del mese di marzo, potete confermare la data di scadenza?",
        "Come posso modificare l'indirizzo di spedizione per il mio prossimo ordine?",
        "Quali documenti sono necessari per completare la registrazione?",
        "Ho bisogno di assistenza per configurare il nuovo dispositivo.",
        "Vorrei sapere se è possibile rateizzare il pagamento dell'ultimo acquisto.",
        "Quali sono gli orari di apertura del centro assistenza di Milano?",
        "Ho una domanda riguardo le specifiche tecniche del prodotto XYZ.",
        "Sto valutando l'acquisto del vostro servizio, quali sono le differenze tra i vari piani?",
        "Ho notato che il mio abbonamento scade il prossimo mese, quali sono le opzioni di rinnovo?",
        "Potete fornirmi il tracking number della mia spedizione?",
        "Vorrei verificare lo stato del mio ordine #12345.",
        "È possibile parlare con un operatore per una consulenza sul prodotto?",
        "Quali sono i tempi di consegna previsti per la mia zona?",
        "Ho bisogno di una copia della mia ultima fattura per la contabilità aziendale.",
        
        # Sentiment negativo
        "Sono estremamente deluso dal vostro servizio, è la terza volta che segnalo lo stesso problema!",
        "Il prodotto che ho ricevuto è difettoso e nessuno risponde alle mie email di reclamo.",
        "È inaccettabile attendere due settimane per una risposta, pessimo servizio clienti!",
        "Ho parlato con tre operatori diversi e ognuno mi ha dato informazioni contrastanti, è assurdo!",
        "La qualità del prodotto è nettamente inferiore a quanto pubblicizzato, mi sento truffato.",
        "Non funziona nulla come dovrebbe, sto valutando seriamente di cambiare fornitore.",
        "Sono molto arrabbiato per come è stata gestita la mia pratica, pretendo un rimborso immediato.",
        "Il vostro servizio clienti è il peggiore che abbia mai incontrato, completamente inutile.",
        "È vergognoso far pagare così tanto per un servizio così scadente!",
        "Ho atteso un tecnico per l'intera giornata e non si è presentato nessuno, senza nemmeno una chiamata.",
        "Questa è l'ultima possibilità che vi do prima di rivolgermi alle associazioni consumatori.",
        "Non comprerò mai più un vostro prodotto, esperienza terribile dall'inizio alla fine.",
        "Il vostro operatore è stato scortese e poco professionale, pretendo delle scuse formali.",
        "Dopo tre tentativi falliti di risolvere il problema, sono completamente frustrato e insoddisfatto.",
        "La connessione continua a cadere nonostante le vostre rassicurazioni che il problema era stato risolto."
    ],
    'sentiment': [
        # 15 positivi
        'positivo', 'positivo', 'positivo', 'positivo', 'positivo',
        'positivo', 'positivo', 'positivo', 'positivo', 'positivo',
        'positivo', 'positivo', 'positivo', 'positivo', 'positivo',
        
        # 15 neutri
        'neutro', 'neutro', 'neutro', 'neutro', 'neutro',
        'neutro', 'neutro', 'neutro', 'neutro', 'neutro',
        'neutro', 'neutro', 'neutro', 'neutro', 'neutro',
        
        # 15 negativi
        'negativo', 'negativo', 'negativo', 'negativo', 'negativo',
        'negativo', 'negativo', 'negativo', 'negativo', 'negativo',
        'negativo', 'negativo', 'negativo', 'negativo', 'negativo'
    ]
}

# Creazione del DataFrame
df = pd.DataFrame(data)

# Visualizzazione delle prime righe
df.head()

In [ ]:
# Esploriamo la distribuzione dei sentiment
plt.figure(figsize=(10, 6))
sns.countplot(y='sentiment', data=df, order=df['sentiment'].value_counts().index)
plt.title('Distribuzione dei Sentiment')
plt.xlabel('Numero di Messaggi')
plt.ylabel('Sentiment')
plt.tight_layout()
plt.show()

## 3. Preprocessing del Testo

Prima di procedere con l'analisi del sentiment, dobbiamo preprocessare i testi. Il preprocessing per l'analisi del sentiment può essere leggermente diverso rispetto ad altri compiti NLP, poiché elementi come la punteggiatura o certe stopwords possono essere indicativi del sentiment.

In [ ]:
# Inizializzazione dello stemmer per l'italiano
stemmer = SnowballStemmer('italian')
stop_words = set(stopwords.words('italian'))

# Rimuoviamo alcune stopwords che possono essere indicative del sentiment
sentiment_stopwords = {'non', 'né', 'no', 'mai', 'poco', 'troppo', 'molto', 'più', 'meno'}
filtered_stop_words = stop_words - sentiment_stopwords

def preprocess_text(text, remove_stopwords=True, stemming=True):
    # Conversione in minuscolo
    text = text.lower()
    
    # Rimozione della punteggiatura (manteniamo ! e ? che possono indicare sentiment)
    text = re.sub(r'[#%&()\*\+/<=>@\[\\\]^_`{|}~]', ' ', text)
    
    # Tokenizzazione
    tokens = word_tokenize(text)
    
    # Rimozione delle stopwords (opzionale)
    if remove_stopwords:
        tokens = [token for token in tokens if token not in filtered_stop_words]
    
    # Stemming (opzionale)
    if stemming:
        tokens = [stemmer.stem(token) for token in tokens]
    
    # Ricostruzione del testo
    processed_text = ' '.join(tokens)
    
    return processed_text

# Applicazione del preprocessing ai testi
df['testo_preprocessato'] = df['testo'].apply(lambda x: preprocess_text(x, remove_stopwords=True, stemming=True))

# Visualizzazione di un esempio prima e dopo il preprocessing
esempio_idx = 0
print(f"Testo originale:\n{df['testo'][esempio_idx]}\n")
print(f"Testo preprocessato:\n{df['testo_preprocessato'][esempio_idx]}")

## 4. Approcci all'Analisi del Sentiment

Esploreremo diversi approcci all'analisi del sentiment, dai più semplici ai più avanzati:

1. **Approcci basati su lessico**: Utilizzano dizionari di parole con punteggi di sentiment predefiniti
2. **Approcci basati su machine learning tradizionale**: Addestrano classificatori su dati etichettati
3. **Approcci basati su deep learning**: Utilizzano modelli pre-addestrati come BERT

### 4.1 Approcci basati su lessico

Iniziamo con approcci basati su lessico, che sono relativamente semplici ma possono essere sorprendentemente efficaci in molti casi.

In [ ]:
# Funzione per tradurre il testo in inglese (necessario per alcuni strumenti di sentiment analysis)
from googletrans import Translator

def translate_to_english(text):
    translator = Translator()
    try:
        translation = translator.translate(text, src='it', dest='en')
        return translation.text
    except Exception as e:
        print(f"Errore nella traduzione: {e}")
        return text

# Traduciamo un sottoinsieme di testi per testare gli strumenti in inglese
# Nota: in un'applicazione reale, potresti voler utilizzare strumenti specifici per l'italiano
sample_indices = [0, 15, 30]  # Un esempio per ogni categoria
for idx in sample_indices:
    print(f"Originale ({df['sentiment'][idx]}): {df['testo'][idx]}")
    translated = translate_to_english(df['testo'][idx])
    print(f"Tradotto: {translated}\n")

In [ ]:
# Utilizziamo VADER per l'analisi del sentiment
# VADER è specificamente progettato per i social media e funziona bene con testi informali
# Nota: VADER è progettato per l'inglese, quindi utilizziamo i testi tradotti

sid = SentimentIntensityAnalyzer()

def analyze_sentiment_vader(text):
    # Traduciamo il testo in inglese
    english_text = translate_to_english(text)
    
    # Otteniamo i punteggi di sentiment
    scores = sid.polarity_scores(english_text)
    
    # Determiniamo la categoria di sentiment
    if scores['compound'] >= 0.05:
        sentiment = 'positivo'
    elif scores['compound'] <= -0.05:
        sentiment = 'negativo'
    else:
        sentiment = 'neutro'
    
    return {
        'sentiment': sentiment,
        'compound': scores['compound'],
        'pos': scores['pos'],
        'neu': scores['neu'],
        'neg': scores['neg']
    }

# Testiamo VADER su alcuni esempi
for idx in sample_indices:
    text = df['testo'][idx]
    true_sentiment = df['sentiment'][idx]
    result = analyze_sentiment_vader(text)
    
    print(f"Testo: {text}")
    print(f"Sentiment reale: {true_sentiment}")
    print(f"Sentiment predetto: {result['sentiment']}")
    print(f"Punteggi: Compound={result['compound']:.4f}, Pos={result['pos']:.4f}, Neu={result['neu']:.4f}, Neg={result['neg']:.4f}\n")

In [ ]:
# Utilizziamo TextBlob per l'analisi del sentiment
# TextBlob offre un'API semplice per compiti NLP comuni
# Nota: TextBlob è progettato principalmente per l'inglese

def analyze_sentiment_textblob(text):
    # Traduciamo il testo in inglese
    english_text = translate_to_english(text)
    
    # Creiamo un oggetto TextBlob
    blob = TextBlob(english_text)
    
    # Otteniamo il punteggio di polarità (-1 a 1)
    polarity = blob.sentiment.polarity
    
    # Determiniamo la categoria di sentiment
    if polarity > 0.1:
        sentiment = 'positivo'
    elif polarity < -0.1:
        sentiment = 'negativo'
    else:
        sentiment = 'neutro'
    
    return {
        'sentiment': sentiment,
        'polarity': polarity,
        'subjectivity': blob.sentiment.subjectivity
    }

# Testiamo TextBlob su alcuni esempi
for idx in sample_indices:
    text = df['testo'][idx]
    true_sentiment = df['sentiment'][idx]
    result = analyze_sentiment_textblob(text)
    
    print(f"Testo: {text}")
    print(f"Sentiment reale: {true_sentiment}")
    print(f"Sentiment predetto: {result['sentiment']}")
    print(f"Punteggi: Polarità={result['polarity']:.4f}, Soggettività={result['subjectivity']:.4f}\n")

### 4.2 Approcci basati su machine learning tradizionale

Ora passiamo agli approcci basati su machine learning, che addestrano classificatori su dati etichettati.

In [ ]:
# Divisione in training e test set
X = df['testo_preprocessato']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Visualizzazione delle dimensioni delle feature
print(f"Dimensioni TF-IDF: {X_train_tfidf.shape}")

In [ ]:
# Funzione per addestrare e valutare un modello
def train_and_evaluate(model, X_train, X_test, y_train, y_test, model_name):
    # Addestramento
    model.fit(X_train, y_train)
    
    # Predizione
    y_pred = model.predict(X_test)
    
    # Valutazione
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print(f"Modello: {model_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(report)
    
    # Matrice di confusione
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=sorted(y.unique()), 
                yticklabels=sorted(y.unique()))
    plt.title(f'Matrice di Confusione - {model_name}')
    plt.ylabel('Valore Reale')
    plt.xlabel('Valore Predetto')
    plt.tight_layout()
    plt.show()
    
    return model, accuracy

In [ ]:
# Modelli da testare
models = {
    'Naive Bayes': MultinomialNB(),
    'Regressione Logistica': LogisticRegression(max_iter=1000, random_state=42),
    'SVM': LinearSVC(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42)
}

# Risultati per confronto
results = []

# Addestramento e valutazione
for name, model in models.items():
    _, accuracy = train_and_evaluate(model, X_train_tfidf, X_test_tfidf, y_train, y_test, name)
    results.append((name, accuracy))

In [ ]:
# Confronto dei risultati
results_df = pd.DataFrame({
    'Modello': [r[0] for r in results],
    'Accuracy': [r[1] for r in results]
})

# Visualizzazione dei risultati
plt.figure(figsize=(12, 6))
sns.barplot(x='Modello', y='Accuracy', data=results_df)
plt.title('Confronto delle Performance dei Modelli')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Tabella dei risultati
results_df

### 4.3 Approcci basati su deep learning

Infine, esploriamo approcci basati su deep learning, utilizzando modelli pre-addestrati come BERT.

In [ ]:
# Utilizziamo un modello pre-addestrato per l'analisi del sentiment
# Nota: questo richiede una connessione internet per scaricare il modello

# Inizializziamo il pipeline di sentiment analysis
# Utilizziamo un modello multilingue che supporta l'italiano
sentiment_analyzer = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

def analyze_sentiment_bert(text):
    # Il modello restituisce un punteggio da 1 (molto negativo) a 5 (molto positivo)
    result = sentiment_analyzer(text)[0]
    score = int(result['label'].split()[0])  # Estrae il numero dal label (es. "5 stars")
    
    # Mappiamo il punteggio alle nostre categorie
    if score >= 4:
        sentiment = 'positivo'
    elif score <= 2:
        sentiment = 'negativo'
    else:
        sentiment = 'neutro'
    
    return {
        'sentiment': sentiment,
        'score': score,
        'confidence': result['score']
    }

# Testiamo il modello BERT su alcuni esempi
for idx in sample_indices:
    text = df['testo'][idx]
    true_sentiment = df['sentiment'][idx]
    result = analyze_sentiment_bert(text)
    
    print(f"Testo: {text}")
    print(f"Sentiment reale: {true_sentiment}")
    print(f"Sentiment predetto: {result['sentiment']}")
    print(f"Punteggio: {result['score']}/5 (Confidenza: {result['confidence']:.4f})\n")

In [ ]:
# Valutiamo il modello BERT sull'intero dataset
def evaluate_bert_model(texts, true_labels):
    predictions = []
    for text in texts:
        result = analyze_sentiment_bert(text)
        predictions.append(result['sentiment'])
    
    accuracy = accuracy_score(true_labels, predictions)
    report = classification_report(true_labels, predictions)
    
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(report)
    
    # Matrice di confusione
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=sorted(set(true_labels)), 
                yticklabels=sorted(set(true_labels)))
    plt.title('Matrice di Confusione - BERT')
    plt.ylabel('Valore Reale')
    plt.xlabel('Valore Predetto')
    plt.tight_layout()
    plt.show()
    
    return predictions, accuracy

# Per risparmiare tempo, valutiamo solo sul test set
bert_predictions, bert_accuracy = evaluate_bert_model(X_test, y_test)

In [ ]:
# Aggiungiamo il risultato di BERT al confronto
results_df = pd.concat([results_df, pd.DataFrame({'Modello': ['BERT'], 'Accuracy': [bert_accuracy]})])

# Visualizzazione dei risultati aggiornati
plt.figure(figsize=(12, 6))
sns.barplot(x='Modello', y='Accuracy', data=results_df)
plt.title('Confronto delle Performance dei Modelli (incluso BERT)')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Tabella dei risultati aggiornata
results_df

## 5. Analisi degli Errori

Analizziamo alcuni esempi in cui il modello ha commesso errori, per comprendere meglio le sue limitazioni.

In [ ]:
# Supponiamo di utilizzare il modello SVM come riferimento
svm_model = LinearSVC(random_state=42)
svm_model.fit(X_train_tfidf, y_train)
y_pred_svm = svm_model.predict(X_test_tfidf)

# Identifichiamo gli errori
errors = []
for i, (true, pred) in enumerate(zip(y_test, y_pred_svm)):
    if true != pred:
        errors.append({
            'testo': df['testo'].iloc[X_test.index[i]],
            'sentiment_reale': true,
            'sentiment_predetto': pred
        })

# Visualizziamo alcuni errori
pd.DataFrame(errors).head(10)

## 6. Applicazione a Nuovi Dati

Ora che abbiamo esplorato diversi approcci, creiamo una funzione che combina i risultati di più modelli per un'analisi del sentiment più robusta.

In [ ]:
# Funzione per l'analisi del sentiment che combina più approcci
def analyze_sentiment_ensemble(text):
    results = {}
    
    # Approccio basato su lessico (VADER)
    vader_result = analyze_sentiment_vader(text)
    results['vader'] = vader_result['sentiment']
    
    # Approccio basato su lessico (TextBlob)
    textblob_result = analyze_sentiment_textblob(text)
    results['textblob'] = textblob_result['sentiment']
    
    # Approccio basato su machine learning (SVM)
    # Preprocessing e trasformazione
    processed_text = preprocess_text(text)
    features = tfidf_vectorizer.transform([processed_text])
    # Predizione
    svm_prediction = svm_model.predict(features)[0]
    results['svm'] = svm_prediction
    
    # Approccio basato su deep learning (BERT)
    bert_result = analyze_sentiment_bert(text)
    results['bert'] = bert_result['sentiment']
    
    # Voto di maggioranza
    predictions = list(results.values())
    from collections import Counter
    counter = Counter(predictions)
    majority_vote = counter.most_common(1)[0][0]
    
    # Dettagli aggiuntivi
    details = {
        'vader_compound': vader_result['compound'],
        'textblob_polarity': textblob_result['polarity'],
        'bert_score': bert_result['score'],
        'individual_predictions': results
    }
    
    return {
        'sentiment': majority_vote,
        'details': details
    }

# Testiamo la funzione ensemble su alcuni esempi
test_texts = [
    "Il vostro servizio clienti è eccellente, ho risolto il problema in pochi minuti!",
    "Vorrei sapere come posso modificare i miei dati personali nel profilo.",
    "Sono molto deluso dalla qualità del prodotto, non funziona come dovrebbe.",
    "Ho provato a contattarvi più volte ma nessuno risponde, pessimo servizio!",
    "La nuova interfaccia è molto più intuitiva, complimenti per il miglioramento."
]

for text in test_texts:
    result = analyze_sentiment_ensemble(text)
    print(f"Testo: {text}")
    print(f"Sentiment (ensemble): {result['sentiment']}")
    print(f"Predizioni individuali: {result['details']['individual_predictions']}")
    print("---")

## 7. Visualizzazione e Interpretazione dei Risultati

Creiamo alcune visualizzazioni utili per interpretare i risultati dell'analisi del sentiment nel contesto del customer care.

In [ ]:
# Simuliamo un dataset di comunicazioni dei clienti nel tempo
import random
from datetime import datetime, timedelta

# Generiamo date per un periodo di 30 giorni
start_date = datetime(2023, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(30)]

# Generiamo sentiment casuali con una distribuzione realistica
sentiments = []
for _ in range(300):  # 300 comunicazioni in totale
    r = random.random()
    if r < 0.25:
        sentiments.append('negativo')
    elif r < 0.65:
        sentiments.append('neutro')
    else:
        sentiments.append('positivo')

# Assegniamo date casuali
random_dates = [random.choice(dates) for _ in range(300)]

# Creiamo il DataFrame
time_data = pd.DataFrame({
    'data': random_dates,
    'sentiment': sentiments
})

# Ordiniamo per data
time_data = time_data.sort_values('data')

# Visualizzazione dell'andamento del sentiment nel tempo
sentiment_counts = time_data.groupby(['data', 'sentiment']).size().unstack().fillna(0)

# Calcoliamo il sentiment netto (positivo - negativo)
sentiment_counts['netto'] = sentiment_counts['positivo'] - sentiment_counts['negativo']

# Visualizziamo l'andamento
plt.figure(figsize=(15, 10))

# Subplot 1: Conteggio per categoria
plt.subplot(2, 1, 1)
sentiment_counts[['positivo', 'neutro', 'negativo']].plot(kind='bar', stacked=True, ax=plt.gca())
plt.title('Distribuzione del Sentiment nel Tempo')
plt.xlabel('Data')
plt.ylabel('Numero di Comunicazioni')
plt.xticks(rotation=45)
plt.legend(title='Sentiment')

# Subplot 2: Sentiment netto
plt.subplot(2, 1, 2)
sentiment_counts['netto'].plot(kind='line', marker='o', ax=plt.gca())
plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
plt.title('Sentiment Netto nel Tempo (Positivo - Negativo)')
plt.xlabel('Data')
plt.ylabel('Sentiment Netto')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Simuliamo dati per diversi canali di comunicazione
channels = ['Email', 'Chat', 'Telefono', 'Social Media', 'App']
channel_data = []

for channel in channels:
    # Generiamo distribuzioni di sentiment diverse per ogni canale
    if channel == 'Email':
        pos, neu, neg = 0.3, 0.4, 0.3
    elif channel == 'Chat':
        pos, neu, neg = 0.4, 0.4, 0.2
    elif channel == 'Telefono':
        pos, neu, neg = 0.25, 0.35, 0.4
    elif channel == 'Social Media':
        pos, neu, neg = 0.2, 0.3, 0.5
    else:  # App
        pos, neu, neg = 0.35, 0.45, 0.2
    
    # Numero di comunicazioni per canale
    n = random.randint(80, 120)
    
    for _ in range(n):
        r = random.random()
        if r < neg:
            sentiment = 'negativo'
        elif r < neg + neu:
            sentiment = 'neutro'
        else:
            sentiment = 'positivo'
        
        channel_data.append({
            'canale': channel,
            'sentiment': sentiment
        })

# Creiamo il DataFrame
channel_df = pd.DataFrame(channel_data)

# Visualizziamo la distribuzione del sentiment per canale
plt.figure(figsize=(12, 8))
sns.countplot(x='canale', hue='sentiment', data=channel_df, palette={'positivo': 'green', 'neutro': 'gray', 'negativo': 'red'})
plt.title('Distribuzione del Sentiment per Canale di Comunicazione')
plt.xlabel('Canale')
plt.ylabel('Numero di Comunicazioni')
plt.legend(title='Sentiment')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Calcoliamo le percentuali di sentiment per canale
channel_percentages = channel_df.groupby(['canale', 'sentiment']).size().unstack().fillna(0)
channel_percentages = channel_percentages.div(channel_percentages.sum(axis=1), axis=0) * 100

# Visualizziamo le percentuali
plt.figure(figsize=(12, 8))
channel_percentages.plot(kind='bar', stacked=True, ax=plt.gca(), 
                         color=['red', 'gray', 'green'])
plt.title('Percentuale di Sentiment per Canale di Comunicazione')
plt.xlabel('Canale')
plt.ylabel('Percentuale')
plt.legend(title='Sentiment')
plt.xticks(rotation=45)

# Aggiungiamo le etichette con le percentuali
for i, channel in enumerate(channel_percentages.index):
    cumulative = 0
    for sentiment in channel_percentages.columns:
        value = channel_percentages.loc[channel, sentiment]
        if value > 5:  # Mostra solo percentuali > 5% per leggibilità
            plt.text(i, cumulative + value/2, f'{value:.1f}%', 
                     ha='center', va='center', color='white', fontweight='bold')
        cumulative += value

plt.tight_layout()
plt.show()

## 8. Implementazione in un Sistema di Customer Care

Vediamo come l'analisi del sentiment potrebbe essere integrata in un sistema di customer care reale.

In [ ]:
# Simuliamo un sistema di prioritizzazione delle richieste basato sul sentiment

# Classe per rappresentare una richiesta di customer care
class CustomerRequest:
    def __init__(self, id, text, channel, timestamp):
        self.id = id
        self.text = text
        self.channel = channel
        self.timestamp = timestamp
        self.sentiment = None
        self.sentiment_score = None
        self.priority = None
    
    def analyze_sentiment(self):
        # Utilizziamo la nostra funzione ensemble
        result = analyze_sentiment_ensemble(self.text)
        self.sentiment = result['sentiment']
        
        # Calcoliamo un punteggio numerico per il sentiment
        if self.sentiment == 'positivo':
            self.sentiment_score = 1
        elif self.sentiment == 'neutro':
            self.sentiment_score = 0
        else:  # negativo
            self.sentiment_score = -1
    
    def calculate_priority(self):
        # Base: più vecchia è la richiesta, più alta è la priorità
        age_hours = (datetime.now() - self.timestamp).total_seconds() / 3600
        age_factor = min(age_hours / 24, 1)  # Normalizzato a 1 dopo 24 ore
        
        # Sentiment: richieste negative hanno priorità più alta
        sentiment_factor = 0
        if self.sentiment == 'negativo':
            sentiment_factor = 0.5
        
        # Canale: alcuni canali potrebbero avere priorità diverse
        channel_factor = 0
        if self.channel == 'Telefono':
            channel_factor = 0.3  # Priorità più alta per telefonate
        elif self.channel == 'Social Media':
            channel_factor = 0.2  # Priorità media per social (visibilità pubblica)
        
        # Calcolo della priorità complessiva (0-1, dove 1 è la massima priorità)
        self.priority = min(age_factor + sentiment_factor + channel_factor, 1)
        
        return self.priority
    
    def __str__(self):
        return f"ID: {self.id}, Canale: {self.channel}, Sentiment: {self.sentiment}, Priorità: {self.priority:.2f}\nTesto: {self.text}"

# Simuliamo alcune richieste
sample_requests = [
    CustomerRequest(1, "Non riesco ad accedere al mio account da giorni e nessuno risponde alle mie email!", "Email", datetime.now() - timedelta(hours=20)),
    CustomerRequest(2, "Vorrei informazioni sui vostri piani tariffari per le aziende.", "Chat", datetime.now() - timedelta(hours=2)),
    CustomerRequest(3, "Il vostro servizio è eccellente, grazie per la rapida risoluzione del mio problema.", "Email", datetime.now() - timedelta(hours=5)),
    CustomerRequest(4, "Ho riscontrato un problema con l'ultimo aggiornamento dell'app, continua a bloccarsi.", "App", datetime.now() - timedelta(hours=10)),
    CustomerRequest(5, "Sono estremamente deluso dal vostro servizio, è la terza volta che mi mandate un prodotto difettoso!", "Telefono", datetime.now() - timedelta(hours=1))
]

# Analizziamo il sentiment e calcoliamo la priorità
for request in sample_requests:
    request.analyze_sentiment()
    request.calculate_priority()

# Ordiniamo le richieste per priorità
prioritized_requests = sorted(sample_requests, key=lambda x: x.priority, reverse=True)

# Visualizziamo le richieste prioritizzate
print("Richieste ordinate per priorità:\n")
for i, request in enumerate(prioritized_requests):
    print(f"{i+1}. {request}\n")

In [ ]:
# Visualizziamo la prioritizzazione
request_data = [{
    'id': r.id,
    'sentiment': r.sentiment,
    'channel': r.channel,
    'age_hours': (datetime.now() - r.timestamp).total_seconds() / 3600,
    'priority': r.priority,
    'text': r.text[:50] + '...' if len(r.text) > 50 else r.text
} for r in sample_requests]

request_df = pd.DataFrame(request_data)

# Visualizziamo i fattori che influenzano la priorità
plt.figure(figsize=(12, 8))

# Creiamo un grafico a dispersione
scatter = plt.scatter(request_df['age_hours'], request_df['priority'], 
                      c=request_df['sentiment'].map({'positivo': 'green', 'neutro': 'gray', 'negativo': 'red'}),
                      s=100, alpha=0.7)

# Aggiungiamo etichette per ogni punto
for i, row in request_df.iterrows():
    plt.annotate(f"ID: {row['id']}\n{row['channel']}", 
                 (row['age_hours'], row['priority']),
                 xytext=(10, 0), textcoords='offset points')

plt.title('Prioritizzazione delle Richieste dei Clienti')
plt.xlabel('Età della Richiesta (ore)')
plt.ylabel('Priorità')
plt.ylim(0, 1.1)

# Aggiungiamo una legenda per il sentiment
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Negativo'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='gray', markersize=10, label='Neutro'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='green', markersize=10, label='Positivo')
]
plt.legend(handles=legend_elements, title='Sentiment')

plt.tight_layout()
plt.show()

## 9. Conclusioni e Considerazioni Finali

In questo notebook, abbiamo esplorato l'analisi del sentiment nel contesto del customer care. Abbiamo:

1. Creato un dataset di esempio di comunicazioni di customer care
2. Preprocessato i testi per l'analisi del sentiment
3. Implementato e confrontato diversi approcci di analisi del sentiment:
   - Approcci basati su lessico (VADER, TextBlob)
   - Approcci basati su machine learning tradizionale (Naive Bayes, SVM, ecc.)
   - Approcci basati su deep learning (BERT)
4. Creato un sistema ensemble che combina più approcci
5. Visualizzato e interpretato i risultati dell'analisi del sentiment
6. Simulato un sistema di prioritizzazione delle richieste basato sul sentiment

### Considerazioni per l'Implementazione Reale

In un contesto aziendale reale, ci sono ulteriori considerazioni da tenere in mente:

- **Specificità del dominio**: I modelli generici di sentiment analysis potrebbero non catturare sfumature specifiche del dominio. Ad esempio, termini tecnici o gergo aziendale potrebbero essere interpretati erroneamente. È consigliabile fine-tuning su dati specifici del dominio.

- **Multilingualità**: Se l'azienda serve clienti in diverse lingue, è importante utilizzare modelli che supportino tutte le lingue rilevanti o implementare soluzioni specifiche per lingua.

- **Contesto più ampio**: Il sentiment di un singolo messaggio potrebbe non raccontare l'intera storia. Considerare la storia delle interazioni precedenti e il contesto più ampio può fornire una comprensione più accurata.

- **Feedback loop**: Implementare un meccanismo per raccogliere feedback dagli operatori sulle analisi del sentiment errate, utilizzando questi dati per migliorare continuamente il modello.

- **Etica e privacy**: Assicurarsi che l'analisi del sentiment sia utilizzata in modo etico e rispettoso della privacy dei clienti. Evitare di fare inferenze invasive o di utilizzare i risultati in modi che potrebbero danneggiare la fiducia dei clienti.

- **Integrazione nei workflow**: L'analisi del sentiment dovrebbe essere integrata nei workflow esistenti in modo da supportare, non sostituire, il giudizio umano. Gli operatori dovrebbero essere formati su come interpretare e utilizzare efficacemente i risultati dell'analisi.

L'analisi del sentiment nel customer care può fornire insights preziosi e migliorare significativamente l'esperienza del cliente, permettendo alle aziende di identificare rapidamente problemi emergenti, prioritizzare le richieste più urgenti e monitorare la soddisfazione generale nel tempo.